In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
device = 'cuda'

In [5]:
model_name = "openai-community/gpt2-medium"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Force float16 instead of BF16
    device_map="auto"           # Auto-detect the best device
)

In [6]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [7]:
sd_hf = model.state_dict()
for k, v in sd_hf.items():
  print(k, v.shape)

transformer.wte.weight torch.Size([50257, 1024])
transformer.wpe.weight torch.Size([1024, 1024])
transformer.h.0.ln_1.weight torch.Size([1024])
transformer.h.0.ln_1.bias torch.Size([1024])
transformer.h.0.attn.c_attn.weight torch.Size([1024, 3072])
transformer.h.0.attn.c_attn.bias torch.Size([3072])
transformer.h.0.attn.c_proj.weight torch.Size([1024, 1024])
transformer.h.0.attn.c_proj.bias torch.Size([1024])
transformer.h.0.ln_2.weight torch.Size([1024])
transformer.h.0.ln_2.bias torch.Size([1024])
transformer.h.0.mlp.c_fc.weight torch.Size([1024, 4096])
transformer.h.0.mlp.c_fc.bias torch.Size([4096])
transformer.h.0.mlp.c_proj.weight torch.Size([4096, 1024])
transformer.h.0.mlp.c_proj.bias torch.Size([1024])
transformer.h.1.ln_1.weight torch.Size([1024])
transformer.h.1.ln_1.bias torch.Size([1024])
transformer.h.1.attn.c_attn.weight torch.Size([1024, 3072])
transformer.h.1.attn.c_attn.bias torch.Size([3072])
transformer.h.1.attn.c_proj.weight torch.Size([1024, 1024])
transformer.h.1

In [8]:
for module in model.modules():
    print(type(module))

<class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>
<class 'transformers.models.gpt2.modeling_gpt2.GPT2Model'>
<class 'torch.nn.modules.sparse.Embedding'>
<class 'torch.nn.modules.sparse.Embedding'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.container.ModuleList'>
<class 'transformers.models.gpt2.modeling_gpt2.GPT2Block'>
<class 'torch.nn.modules.normalization.LayerNorm'>
<class 'transformers.models.gpt2.modeling_gpt2.GPT2Attention'>
<class 'transformers.pytorch_utils.Conv1D'>
<class 'transformers.pytorch_utils.Conv1D'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.normalization.LayerNorm'>
<class 'transformers.models.gpt2.modeling_gpt2.GPT2MLP'>
<class 'transformers.pytorch_utils.Conv1D'>
<class 'transformers.pytorch_utils.Conv1D'>
<class 'transformers.activations.NewGELUActivation'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'transformers.models.gpt2.modeling_gpt2.GPT2Block'>